In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using JuliaDB, TaylorSeries, PlanetaryEphemeris

In [ ]:
using JLD

In [ ]:
using Apophis

In [ ]:
using Statistics

In [ ]:
using LinearAlgebra

In [ ]:
using Dates

In [ ]:
#using Plots

In [ ]:
using Gaston

In [ ]:
using StatsBase

In [ ]:
using DelimitedFiles

In [ ]:
dq = set_variables("δx", order=5, numvars=7)

In [ ]:
Apophis.loadjpleph()

In [ ]:
radec_0420 = JuliaDB.load("radec_2004_2020_o5v7.jdb")
radec_2021 = JuliaDB.load("radec_2020_2021_o5v7.jdb")
radec_0421_ = merge(radec_0420, radec_2021)
radec_0421 = transform(radec_0421_, :σ => w8sveres17.(radec_0421_))

In [ ]:
deldop_0513 = JuliaDB.load("deldop_2005_2013_o5v7_tord10_niter5.jdb") #deldop_2005_2013_o5v7_ad_tord10.jdb
deldop_2121 = JuliaDB.load("deldop_2021_o5v7_tord10_niter5.jdb") #deldop_2021_o5v7_ad_tord15.jdb
deldop_0521 = merge(deldop_0513, deldop_2121)
;

In [ ]:
del_0513 = filter(x->x.delay_index, deldop_0513)
dop_0513 = filter(x->x.doppler_index, deldop_0513)

del_2121 = filter(x->x.delay_index, deldop_2121)
dop_2121 = filter(x->x.doppler_index, deldop_2121)

del_0521 = filter(x->x.delay_index, deldop_0521)
dop_0521 = filter(x->x.doppler_index, deldop_0521)
;

In [ ]:
res_τ = select(del_0521, (:τ_obs, :τ_comp)=>x->x[1]-x[2])
res_ν = select(dop_0521, (:ν_obs, :ν_comp)=>x->x[1]-x[2])

σ_del_0513 = select(del_0513, :σ_τ)
σ_dop_0513 = select(dop_0513, :σ_ν)

σ_del_2121 = select(del_2121, :σ_τ)
σ_dop_2121 = select(dop_2121, :σ_ν)

σ_del_0521 = select(del_0521, :σ_τ)
σ_dop_0521 = select(dop_0521, :σ_ν)

w_τ = select(del_0521, :σ_τ=>x->1/x^2)
w_ν = select(dop_0521, :σ_ν=>x->1/x^2)


# only Vokrouhlicky et al. (2015) data
res_τ_v15 = select(del_0513, (:τ_obs, :τ_comp)=>x->x[1]-x[2])
res_ν_v15 = select(dop_0513, (:ν_obs, :ν_comp)=>x->x[1]-x[2])

w_τ_v15 = select(del_0513, :σ_τ=>x->1/x^2)
w_ν_v15 = select(dop_0513, :σ_ν=>x->1/x^2)
;

In [ ]:
# check if all RA/Dec observations are CCD:

all(select(radec_0421, :j2000=>x->x == "C" || x == "X"))

In [ ]:
filter(x->x.catalog == " ", radec_0421) # there are 452 RA/Dec obs without catalog info

In [ ]:
#select(radec_0420, :dt_utc_obs)[1] > DateTime(2004)
#filter(x->x.dt_utc_obs<DateTime(2021,1,14), radec_0420)

In [ ]:
#Γ_OR7

In [ ]:
res0 = radec_04XX[1];

In [ ]:
#res0.

In [ ]:
function chisqobs(row, Γ_β, x, outlier::Bool=false)
    A_α_TN = TaylorSeries.gradient(row.α_obs-row.α_corr-row.α_comp)
    A_δ_TN = TaylorSeries.gradient(row.δ_obs-row.δ_corr-row.δ_comp)
    A_i = hcat(A_α_TN(x), A_δ_TN(x))
    outlier_sign = outlier*2-1
    (A_i')*Γ_β*A_i
end

In [ ]:
chisqobs(res0, Γ_OR7, x_OR7)

In [ ]:
#radec_04XX = radec_0415
#radec_04XX = radec_0420
#radec_04XX = radec_0421
radec_04XX = filter(x->( DateTime(x.yr, x.month, x.day) != DateTime(2021, 1, 28) ), radec_0421)

#radec_04XX = filter(x->x.dt_utc_obs<DateTime(2021,1,14), radec_0420)

χ2_rej = 4.0 #4.0 #4.25 #5.025 # Brozovic et al (2018) use Chesley et al (2010) outlier rejection, in turn from Carpino et al (2003)

#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp())^2 + (x.δ_obs-x.δ_corr-x.δ_comp())^2)<χ2_rej, radec_04XX)
#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp(x_OR7))^2 + (x.δ_obs-x.δ_corr-x.δ_comp(x_OR7))^2)/(x.σ^2) ≤ χ2_rej, radec_04XX)

In [ ]:
select(radec_04XX, :dt_utc_obs)

In [ ]:
# Vokrouhlicky et al. (2015) obs table
radec_vok15 = radec_table("../data/vokr15_mpc_formatted.dat")
# vector of RA values from Vokrouhlicky et al. (2015) observations (used for filtering)
vok15_rav = select(radec_vok15, :α_obs);

vok15_ind_filt = findall(x->x ∈ vok15_rav, select(radec_04XX, :α_obs))
@show length(vok15_ind_filt)
;

In [ ]:
# Vorkouhliky et al. (2015) RA/Dec table
radec_v15 = filter(x->x.α_obs ∈ vok15_rav, radec_04XX);

In [ ]:
# RA/Dec debias corrections using table from Farnocchia et al. (2015)
radec_v15_radec_corr_2014 = select(radec_table("../data/vokr15_mpc_formatted.dat", debias_table="2018"), (:α_corr, :δ_corr));

In [ ]:
res_α = select(radec_04XX, (:α_obs, :α_corr, :α_comp)=>x->(x[1]-x[2]-x[3]))
res_δ = select(radec_04XX, (:δ_obs, :δ_corr, :δ_comp)=>x->(x[1]-x[2]-x[3]))

res_α_v15 = select(radec_v15, (:α_obs, :α_comp)=>x->(x[1]-x[2])) - select(radec_v15_radec_corr_2014, :α_corr)
res_δ_v15 = select(radec_v15, (:δ_obs, :δ_comp)=>x->(x[1]-x[2])) - select(radec_v15_radec_corr_2014, :δ_corr)
;

In [ ]:
# read astrometric errors from Tholen et al. (2013)
tho13_errors = readdlm("../data/tholenetal2013_opterror.dat", ',');
# compute weights
w_α_tho13 = 1 ./ (tho13_errors[:,1].^2 .+ tho13_errors[:,3].^2 .+ tho13_errors[:,5].^2)
w_δ_tho13 = 1 ./ (tho13_errors[:,2].^2 .+ tho13_errors[:,4].^2 .+ tho13_errors[:,6].^2)

# Tholen et al. (2013) obs table
radec_tho13 = radec_table("../data/tholen13_mpc_formatted.dat")
# vector of RA values from Tholen et al. (2013) observations (used for filtering)
tho13_rav = select(radec_tho13, :α_obs);

tho13_ind_filt = findall(x->x ∈ tho13_rav, select(radec_04XX, :α_obs))
tho13_ind_filt_v15 = findall(x->x ∈ tho13_rav, select(radec_v15, :α_obs))
@show length(tho13_ind_filt), length(tho13_ind_filt_v15)
;

In [ ]:
function w8sfarnocchia15(row::NamedTuple)
    return w8sfarnocchia15(row.obscode, row.dt_utc_obs, row.catalog)
end
function w8sfarnocchia15(obscode::Union{Int,String}, dt_utc_obs::DateTime, catalog::String)
    w = 1.0 # unit weight (arcseconds)
    if obscode == "H01" # Magdalena Ridge
        return 0.3w
    elseif obscode == "568" # Mauna Kea (MPEC 2014-R71)
        return 0.13w #0.2w #0.2":Verevs17 weight;0.13":Farnocchia15 weight
    elseif obscode == "F51" # PanSTARRS-PS1
        return 0.15w #0.2w #0.2":Verevs17 weight;0.15":Farnocchia15 weight
    else
        return w
    end
end


In [ ]:
# for each observation batch, count the number of observations made in the same night (?) by the same observatory
date_obscode_v = select(radec_04XX, (:dt_utc_obs,:obscode)=>x->(Date(x[1]),x[2]))
date_obscode_v_v15 = select(radec_v15, (:dt_utc_obs,:obscode)=>x->(Date(x[1]),x[2]))
Nv = [count(x->x[1]==i[1] && x[2]==i[2], date_obscode_v) for i in date_obscode_v]
Nv_v15 = [count(x->x[1]==i[1] && x[2]==i[2], date_obscode_v_v15) for i in date_obscode_v_v15]
relax_factor = map(x->x>4.0 ? x/4.0 : 1.0, Nv)
relax_factor_v15 = map(x->x>4.0 ? x/5.0 : 1.0, Nv_v15) # map(x->x>2.0 ? x : 1.0, Nv_v15)

# statistical weights according to Veres et al (2017)
σ_v17 = w8sveres17.(radec_04XX)
σ_v17_v15 = w8sfarnocchia15.(radec_v15) #w8sveres17.(radec_v15)

w_α = 1 ./ (σ_v17.^2)
w_δ = 1 ./ (σ_v17.^2)

w_α_v15 = 1 ./ (σ_v17_v15.^2)
w_δ_v15 = 1 ./ (σ_v17_v15.^2)

# Tholen et al. (2013) astrometry uncertainties
w_α[tho13_ind_filt] .= w_α_tho13
w_δ[tho13_ind_filt] .= w_δ_tho13
w_α_v15[tho13_ind_filt_v15] .= w_α_tho13
w_δ_v15[tho13_ind_filt_v15] .= w_δ_tho13

w_α = w_α./(relax_factor)
w_δ = w_δ./(relax_factor)

w_α_v15 = w_α_v15./(relax_factor_v15)
w_δ_v15 = w_δ_v15./(relax_factor_v15)
;

In [ ]:
length(w_τ), length(w_ν), length(w_α), length(w_δ)

In [ ]:
length(w_τ_v15), length(w_ν_v15), length(w_α_v15), length(w_δ_v15)

In [ ]:
nobs_R = length(del_0521) + length(dop_0521)
nobs_O = length(radec_04XX)
nobs = nobs_R + nobs_O
@show nobs_O, nobs_R, nobs
;

In [ ]:
res_O = vcat(res_α, res_δ)
w_O = vcat(w_α, w_δ)

res_R = vcat(res_τ, res_ν)
w_R = vcat(w_τ, w_ν)

res_OR = vcat(res_R, res_O)
w_OR = vcat(w_R, w_O)
;

In [ ]:
# only Vokrouhliky et al. (2015) data
res_O_v15 = vcat(res_α_v15, res_δ_v15)
w_O_v15 = vcat(w_α_v15, w_δ_v15)

res_R_v15 = vcat(res_τ_v15, res_ν_v15)
w_R_v15 = vcat(w_τ_v15, w_ν_v15)

res_OR_v15 = vcat(res_R_v15, res_O_v15)
w_OR_v15 = vcat(w_R_v15, w_O_v15)
;

In [ ]:
x_OR7_v15, Γ_OR7_v15 = newtonls(res_OR_v15, w_OR_v15, zeros(7), 9)
σ_OR7_v15 = sqrt.(diag(Γ_OR7_v15))
@show x_OR7_v15 σ_OR7_v15 x_OR7_v15[end]/σ_OR7_v15[end]

x_OR6_v15, Γ_OR6_v15 = newtonls_6v(res_OR_v15, w_OR_v15, zeros(7))
σ_OR6_v15 = sqrt.(diag(Γ_OR6_v15))
@show x_OR6_v15 σ_OR6_v15 x_OR6_v15[end]/σ_OR6_v15[end]

In [ ]:
x_OR7, Γ_OR7 = newtonls(res_OR, w_OR, zeros(7))
σ_OR7 = sqrt.(diag(Γ_OR7))
@show x_OR7 σ_OR7 x_OR7[end]/σ_OR7[end]

x_OR6, Γ_OR6 = newtonls_6v(res_OR, w_OR, zeros(7))
σ_OR6 = sqrt.(diag(Γ_OR6))
@show x_OR6 σ_OR6 x_OR6[end]/σ_OR6[end]

x_O7, Γ_O7 = newtonls(res_O, w_O, zeros(7))
σ_O7 = sqrt.(diag(Γ_O7))
@show x_O7 σ_O7 x_O7[end]/σ_O7[end]

x_R7, Γ_R7 = newtonls(res_R, w_R, vcat(zeros(6),10))#zeros(7))
σ_R7 = sqrt.(diag(Γ_R7))
@show x_R7 σ_R7 x_R7[end]/σ_R7[end]

x_O6, Γ_O6 = newtonls(res_O, w_O, zeros(7))
σ_O6 = sqrt.(diag(Γ_O6))
@show x_O6 σ_O6 x_O6[end]/σ_O6[end]

x_R6, Γ_R6 = newtonls(res_R, w_R, vcat(zeros(6),10))#zeros(7))
σ_R6 = sqrt.(diag(Γ_R6))
@show x_R6 σ_R6 x_R6[end]/σ_R6[end]
;

In [ ]:
@show x_OR7 Γ_OR7

In [ ]:
@show x_OR7.*vcat(1e-8ones(6), 1e-14)

# $\Gamma$ matrix eigenvalues/vectors

In [ ]:
Hermitian(Γ_OR7) - Γ_OR7

In [ ]:
norm(Hermitian(Γ_OR7) - Γ_OR7)

In [ ]:
Γ_OR7_cholf = cholesky(Hermitian(Γ_OR7))

In [ ]:
Γ_OR7_cU = Γ_OR7_cholf.U
Γ_OR7_cL = Γ_OR7_cholf.L

In [ ]:
# Compute nominal initial condition in au, au/day, au/day^2
q0_OR7 = [
    -0.18034747703273316,
    0.9406910666200128,
    0.3457360259054398,
    -0.016265942170279046,
    4.392889725556651e-5,
    -0.00039519931615139716,
    0.0
] .+ vcat(1e-8ones(6), 1e-14).*x_OR7

In [ ]:
# relative scalings analogous to Milani (2005)
scaling_milani_2005 = vcat(  (1/norm(q0_OR7[1:3]))*ones(3), (1/norm(q0_OR7[4:6]))*ones(3), 1/norm(q0_OR7[7])  )

# jet transport propagation scaling
jt_scaling = vcat(1e-8ones(6), 1e-14)
;

In [ ]:
# compute cov matrix with relative scalings
Γ_OR7_scm05 = diagm(scaling_milani_2005) * ( diagm(jt_scaling)*Γ_OR7*diagm(jt_scaling) ) * diagm(scaling_milani_2005)

In [ ]:
# compute normal matrix with relative scalings
C_OR7_scm05 = inv(Γ_OR7_scm05)

In [ ]:
# compute eigenvalues, eigenvectors of normal matrix with relative scalings
C_eig_OR7_scm05 = eigen(C_OR7_scm05)

In [ ]:
# k₁ * V₁ = 1/√λ₁ * V₁
k1_V1 = C_eig_OR7_scm05.vectors[:,1]/sqrt(C_eig_OR7_scm05.values[1])

In [ ]:
# revert eigenvector of "weak" direction to JT scaling
( k1_V1 ./ scaling_milani_2005 ) ./ jt_scaling

In [ ]:
Taylor1(10) * ( k1_V1 ./ scaling_milani_2005 )

In [ ]:
@show ( ( k1_V1 ./ scaling_milani_2005 )  )  #./ jt_scaling

In [ ]:
σ_OR7

In [ ]:
@show x_OR7 .* jt_scaling

In [ ]:
#@show Γ_OR7

# $\chi^2$, RMS

The gravity-only solution post-fit normalized RMS for the optical, radar, and combined optical and radar astrometry are, respectively, 0.448, 14.720 and 1.0259. Moreover, the gravity-only solution post-fit right-ascension weighted mean residual is 0.072$~$arcsec, while the declination weighted mean residual is 0.218$~$arcsec. The weighted time-delay and Doppler shift mean residual for the gravity-only solution are, respectively, -0.06 us and 0.043 Hz. On the other hand, the non-gravitational solution normalized RMS for the optical, radar, and combined optical and radar astrometry are, respectively, 0.313, 0.387 and 0.314. The right-ascension, declination, time-delay and Doppler shift weighted mean residuals, are, respectively, 0.008$~$arcsec, 0.003$~$arcsec, -0.0003$~$us and -0.31$~$Hz.

In [ ]:
mean(res_α(x_OR6), weights(w_α)), mean(res_δ(x_OR6), weights(w_δ))

In [ ]:
mean(res_α(x_OR7), weights(w_α)), mean(res_δ(x_OR7), weights(w_δ))

In [ ]:
mean(res_τ(x_OR6), weights(w_τ)), mean(res_ν(x_OR6), weights(w_ν))

In [ ]:
mean(res_τ(x_OR7), weights(w_τ)), mean(res_ν(x_OR7), weights(w_ν))

In [ ]:
nrms(res_O(x_OR7), w_O), nrms(res_R(x_OR7), w_R), nrms(res_OR(x_OR7), w_OR)

In [ ]:
nrms(res_O(x_OR6), w_O), nrms(res_R(x_OR6), w_R), nrms(res_OR(x_OR6), w_OR)

In [ ]:
chi2(res_OR(x_OR7), w_OR)

In [ ]:
chi2(res_OR(x_OR6), w_OR)

In [ ]:
(13113.028426000146-1227.769395635357)/13113.028426000146

In [ ]:
x_OR7[end]/σ_OR7[end]

In [ ]:
jd0 = datetime2julian(DateTime(2020,12,17)) #Julian date (TDB) of integration initial time
et0 = (jd0 - J2000)*daysec

In [ ]:
q00 = [-0.18034747703273316, 0.9406910666200128, 0.3457360259054398, -0.016265942170279046, 4.392889725556651e-5, -0.00039519931615139716]

In [ ]:
q00 .+ (vcat(1e-8*ones(6), 1e-14).*x_OR7)[1:6]

In [ ]:
xas = q00 .- Apophis.kmsec2auday(Apophis.sun_pv(et0)) # .+ (vcat(1e-8*ones(6), 1e-14).*x_OR7)[1:6]

In [ ]:
ϵ0_deg = 84381.448/3600;
m_eq2ecl = Rx(deg2rad(ϵ0_deg));
m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl));

In [ ]:
#kep0 = vcat([pv2kep(xas .+ (vcat(1e-8*ones(6), 1e-14).*dq)[1:6], μ[1], jd0)...][1:6],1e-14(x_OR7[7] + dq[7]));
kep0 = vcat([pv2kep(  m_xv_eq2ecl*(xas .+ (vcat(1e-8*ones(6), 1e-14).*dq)[1:6])  , μ[1], jd0)...][1:6],1e-14(dq[7])); # x_OR6 +
kep0()

In [ ]:
kep_OR7 = kep0(x_OR7)

In [ ]:
orbel_OR7_TN = kep_OR7 .+ dq

In [ ]:
var_names_str = prod(get_variable_names().*" ")
vars_old = get_variables()
vars_new = set_variables(eltype(vars_old[1]), var_names_str[1:prevind(var_names_str, end)], order=vars_old[1].order)

In [ ]:
x_OR7[7], σ_OR7[7]

In [ ]:
ady_kep_au_My = 1e6yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) # au/Myr

In [ ]:
ady_kep_m_y = 1e3au*yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) #m/yr

In [ ]:
t_car2kep(x) = TaylorSeries.jacobian(kep0, x)'
t_kep2ady(x, ady_kep) = TaylorSeries.gradient(ady_kep)(x)

In [ ]:
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7))

In [ ]:
1e4sqrt( (t_kep2ady(kep_OR7, ady_kep_au_My)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_au_My) )

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_m_y)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_m_y) )

In [ ]:
t_car2kep(x_OR7)

In [ ]:
σ_OR7_kep = sqrt.(diag(Γ_OR7_kep))

In [ ]:
# WARNING: these are ecliptic elements!!! also, not sure about the order in the Ω ↔ ω angles
kep_JPL204 = [.1915088664476708, .7458530510952959, 2459101.040925883049, 204.04198656167, 126.6539656076709, 3.336773294259109, -2.886877799607672E-14]
σ_kep_JPL204 = [2.2197e-09, 2.2003e-09, 1.2886e-06, 1.4045e-05, 1.4205e-05, 2.4295e-07, 3.039E-16]
;

In [ ]:
kep_OR7

In [ ]:
kep_JPL204

In [ ]:
σ_kep_JPL204

In [ ]:
σ_OR7_kep

In [ ]:
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7));
Γ_OR6_kep = (t_car2kep(x_OR6)[1:6,1:6]')*Γ_OR6*t_car2kep(x_OR6)[1:6,1:6];

In [ ]:
jd0

In [ ]:
for (x, Γ_kep, sol_str) in [(x_OR6, Γ_OR6_kep, "OR6"), (x_OR7, Γ_OR7_kep, "OR7")]
    println("* ----===OOO===--- *", sol_str)
    @show kep0(x) sqrt.(diag(Γ_kep))
end

In [ ]:
#OR6

0.19150873(117)     \pm 1.52\times 10^{-9}
0.74585318(404)     \pm 1.48\times 10^{-9}
2459101.040828(880) \pm 8.58\times 10^{-7}
204.0423(481)       \pm 1.12\times 10^{-5}
126.6535(068)       \pm 1.12\times 10^{-5}
3.336763(529)       \pm 1.91\times 10^{-7}
 0.0 (fixed)                              

# OR7

0.19150886(705)    \pm 1.96\times 10^{-9}
0.74585305(045)    \pm 1.91\times 10^{-9}
2459101.04092(533) \pm 1.23\times 10^{-6}
204.0419(913)      \pm 1.16\times 10^{-5}
126.6539(606)      \pm 1.19\times 10^{-5}
3.336773(194)      \pm 2.10\times 10^{-7}
-2.8(951)          \pm 2.65\times 10^{-2}

In [ ]:
for (x, Γ, sol_str) in [(x_OR6, Γ_OR6, "OR6"), (x_OR7, Γ_OR7, "OR7")]
    println("* ----===OOO===--- *", sol_str)
    σ_sol_ = sqrt.(diag(Γ))
    σ_sol = length(σ_sol_)==6 ? 1e-8σ_sol_[1:6] : vcat(1e-8σ_sol_[1:6], 1e-14σ_sol_[7])
    @show vcat(q00, 0).+vcat(1e-8x[1:6], 1e-14x[7]) σ_sol
end

In [ ]:
#OR6

-0.1803479(176)            \pm 1.15\times 10^{-8} 
 0.94069120(813)           \pm 2.28\times 10^{-9} 
 0.34573632(688)           \pm 5.61\times 10^{-9} 
-0.0162659364(547)         \pm 4.56\times 10^{-11}
 4.3919(056)\times 10^{-5} \pm 1.17\times 10^{-10}
-0.000395207(186)          \pm 1.97\times 10^{-10}
 0.0 (fixed)                                      

#OR7

-0.1803482(842)            \pm 1.20\times 10^{-8} 
 0.94069105(964)           \pm 2.66\times 10^{-9} 
 0.34573599(054)           \pm 6.41\times 10^{-9} 
-0.0162659397(882)         \pm 5.46\times 10^{-11}
 4.3915(493)\times 10^{-5} \pm 1.26\times 10^{-10}
-0.0003(952)               \pm 2.00\times 10^{-10}
-2.8(951)\times 10^{-14}   \pm 2.66\times 10^{-16}


In [ ]:
res_α_OR6 = res_α(x_OR6)
res_δ_OR6 = res_δ(x_OR6)

res_α_OR7 = res_α(x_OR7)
res_δ_OR7 = res_δ(x_OR7)
;

In [ ]:
mean(res_α_OR6), std(res_α_OR6), mean(res_δ_OR6), std(res_δ_OR6)

In [ ]:
mean(res_α_OR7), std(res_α_OR7), mean(res_δ_OR7), std(res_δ_OR7)

In [ ]:
#bins_hist = -2.0:0.1:2.0
nbins_plot = 100

#h_a = fit(Histogram, res_α_OR6, bins_hist)
h_a = fit(Histogram, res_α_OR6, nbins=nbins_plot)
bc_a = midpoints(h_a.edges[1]) # bin centers
bs_a = fill(h_a.edges[1].step.hi, length(bc_a)) # bin sizes
;

#h_b = fit(Histogram, res_δ_OR6, bins_hist)
h_b = fit(Histogram, res_δ_OR6, nbins=nbins_plot)
bc_b = midpoints(h_b.edges[1]) # bin centers
bs_b = fill(h_b.edges[1].step.hi, length(bc_b)) # bin sizes

#h_c = fit(Histogram, res_α_OR7, bins_hist)
h_c = fit(Histogram, res_α_OR7, nbins=nbins_plot)
bc_c = midpoints(h_c.edges[1]) # bin centers
bs_c = fill(h_c.edges[1].step.hi, length(bc_c)) # bin sizes

#h_d = fit(Histogram, res_δ_OR7, bins_hist)
h_d = fit(Histogram, res_δ_OR7, nbins=nbins_plot)
bc_d = midpoints(h_d.edges[1]) # bin centers
bs_d = fill(h_d.edges[1].step.hi, length(bc_d)) # bin sizes
;

In [ ]:
minimum(res_α_OR6), maximum(res_α_OR6)

In [ ]:
minimum(res_δ_OR6), maximum(res_δ_OR6)

In [ ]:
minimum(res_α_OR7), maximum(res_α_OR7)

In [ ]:
minimum(res_δ_OR7), maximum(res_δ_OR7)

In [ ]:
mean(res_α_OR6), mean(res_δ_OR6)

In [ ]:
mean(res_α_OR7), mean(res_δ_OR7)

In [ ]:
# use findall instead of filter, since we're looking for the indices of ZTF observations
ztf_obs_indx = findall(x->x=="I41", select(radec_04XX, :obscode))

In [ ]:
res_α_OR7[ztf_obs_indx]

In [ ]:
res_δ_OR7[ztf_obs_indx]

In [ ]:
sqrt(  mean(res_δ_OR7[ztf_obs_indx].^2 .+ res_α_OR7[ztf_obs_indx].^2)  )

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv)

cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_OR6, res_δ_OR6)))
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_OR7, res_δ_OR7)))

ell_3σ_x_OR6 = mean(res_α_OR6) .+ cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = mean(res_δ_OR6) .+ cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = mean(res_α_OR7) .+ cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = mean(res_δ_OR7) .+ cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
gap = 0.01
xlim_plt = 3.5
ymin_plt = -4.0
ymax_plt = 5.0

Gaston.set(preamble="set grid; set style circle radius 0.02")

p1 = Gaston.plot(
    bc_a,
    h_a.weights./2, 
    supp=[bs_a./2 h_a.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4", 
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.99; set bmargin at screen 0.8+$gap",
        xrange = (-xlim_plt, xlim_plt),
        format = "''"
    ),
    handle = 1
)
Gaston.plot!(
    bc_c,
    h_c.weights./2, 
    supp=[bs_c./2 h_c.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4"
)

p2 = Gaston.plot(
    res_α_OR6, 
    res_δ_OR6, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'gravity-only'",
    Axes(
        #linetype = :tab10,
        key="at screen 0.835, screen 0.78 box spacing 1.2 height 0.4",  #"outside top right box spacing 1.2 height 1",
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.8; set bmargin at screen 0.1",
        xrange = (-xlim_plt, xlim_plt),
        yrange = (ymin_plt, ymax_plt),
        format = "\"%g\"",
        xlabel="'\$\\alpha\\cos(\\delta)\$ \$[\$arcsec\$]\$'",
        ylabel="'\$\\delta\$ \$[\$arcsec\$]\$'"
    ), 
    handle = 2
)
Gaston.plot!(res_α_OR7, res_δ_OR7, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'non-grav.'"
)
Gaston.plot!([mean(res_α_OR6)], [mean(res_δ_OR6)], curveconf="w p pt 4 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR6)'
Gaston.plot!([mean(res_α_OR7)], [mean(res_δ_OR7)], curveconf="w p pt 6 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR7)'
Gaston.plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, curveconf="w l lw 8 lc 3")
Gaston.plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, curveconf="w l lw 8 lc 4")

p4 = Gaston.plot(
    h_b.weights./2,
    bc_b,
    supp=[h_b.weights./2 bs_b./2],
    curveconf="w boxxy notit fs transparent solid 0.4",
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.85+$gap; set rmargin at screen 0.99; set tmargin at screen 0.8; set bmargin at screen 0.1",
        yrange = (ymin_plt, ymax_plt),
        format = "''"
    ),
    handle = 4
)
Gaston.plot!(
    h_d.weights./2,
    bc_d,
    supp=[h_d.weights./2 bs_d./2],
    curveconf="w boxxy notit fs transparent solid 0.4"
)

Gaston.plot([p1 nothing ; p2 p4])

#Gaston.save(term = "cairolatex", output = "radec_hist.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',14'")
#run(`pdflatex radec_hist.tex`)

In [ ]:
length(res_α)

In [ ]:
mean(w_α.*res_α(x_OR7)), std(w_α.*res_α(x_OR7))

In [ ]:
mean(w_δ.*res_δ(x_OR7)), std(w_δ.*res_δ(x_OR7))

In [ ]:
mean(res_τ(x_OR7)), std(res_τ(x_OR7))

In [ ]:
mean(res_ν(x_OR7)), std(res_ν(x_OR7))

## OR6

In [ ]:
mean(res_τ(x_OR6)), std(res_τ(x_OR6))

In [ ]:
mean(res_ν(x_OR6)), std(res_ν(x_OR6))

In [ ]:
ind_del_0506 = 1:2
ind_del_1213 = 3:17
ind_del_2121 = 18:19

ind_dop_0506 = 1:5
ind_dop_1213 = 6:29
ind_dop_2121 = 30:30
;

In [ ]:
del_index = findall(x->x, select(deldop_0521, :delay_index))
dop_index = findall(x->x, select(deldop_0521, :doppler_index))
length(del_index), length(dop_index)

In [ ]:
#deldop_index = findall(x->x, select(deldop_0521, (:delay_index, :doppler_index)=>(x,y)->x*y))
#select(deldop_0521, (:delay_index, :doppler_index)=>x->x[1]*x[2])
length(deldop_0521)

In [ ]:
del_dates_plot_ = select(filter(x->x.delay_index, deldop_0521), :dt_utc_obs)
dop_dates_plot_ = select(filter(x->x.doppler_index, deldop_0521), :dt_utc_obs)

del_dates_plot = Dates.format.(del_dates_plot_, "dd/mm/yy HHMM")
dop_dates_plot = Dates.format.(dop_dates_plot_, "dd/mm/yy HHMM")

length(del_dates_plot), length(dop_dates_plot)

In [ ]:
#print(prod(string.(select(deldop_0521, :rcvr)).*"\n"))
#print(prod(string.(map(x->x==251 ? "Arecibo" : "Goldstone", select(deldop_0521, :rcvr))).*"\n"))

In [ ]:
p1 = Gaston.plot(
    del_index,
    res_τ(x_OR6),
    supp = σ_del_0521,
    key="box spacing 1.2 height 0.4", #at screen 0.835, screen 0.78 
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set bmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Time-delay resid. \$[\$us\$]\$'",
        xrange = (0,38),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-7,8),
        label="1 '(a)' at graph 0.05,0.9"#,
        #title = "'gravity-only'"
    ),
    handle = 1
)
Gaston.plot!(
    del_index,
    res_τ(x_OR7),
    supp = σ_del_0521,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)

p2 = Gaston.plot(
    dop_index,
    res_ν(x_OR6),
    supp = σ_dop_0521,
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    key="box spacing 1.2 height 0.4",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set tmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Doppler shift resid. \$[\$Hz\$]\$'",
        xrange = (0,38),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-0.6,0.8),
        label="1 '(b)' at graph 0.05,0.9" #,
        # title = "'non-gravitational'"
    ),
    handle = 2
)
Gaston.plot!(
    dop_index,
    res_ν(x_OR7),
    supp = σ_dop_0521,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)


Gaston.plot([p1; p2])

#Gaston.save(term = "cairolatex", output = "deldop_residuals.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex deldop_residuals.tex`)

In [ ]:
deldop_dates_plot_xaxis = vcat("01/06/04 0000", del_dates_plot, "31/03/21 2359");

In [ ]:
tmargin_bmargin_gap = 0.03
bmargin_ths_plt = 0.5 + tmargin_bmargin_gap
tmargin_ths_plt = 0.965
axsbrk_v = 1.25*0.01
axsbrk_h = 1.25*0.005
lw_ths_plt = 3
rmargin_ths_plt = 0.3

p1 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_0521, #[ind_del_0506]
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf = "set rmargin at screen 0.3; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = false, #"format '' time (\"01/01/05\",\"01/01/06\")", # '%d/%b/%y'
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ylabel = "'Time-delay resid. (O-C) [\$\\mu\$s]'",
        border = "1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange = "[-8.0:6.0]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(a)' at graph 0.2,0.9",
        grid=false
        #arrow = "from screen 0.3-0.01,1.0-0.01 to screen 0.3+0.01,1.0+0.01 nohead"
        #format = "x ''"
        #title = "'Time-delay resid. (O-C)'"
    ),
    handle = 1
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_0521, #[ind_del_0506]
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p2 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_0521, #[ind_del_1213]
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.35; set tmargin at screen $tmargin_ths_plt; set rmargin at screen 0.8; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")", # '%d/%b/%y'
        xrange = "[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 2
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_0521, #[ind_del_1213]
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p3 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_0521, #[ind_del_2121]
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.85; set tmargin at screen 0.965; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"03/03/21\",\"11/03/21\")", #'%d/%b/%y'
        xrange = "[\"02/03/21\":\"12/03/21\"]",
        #xlabel = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4+8",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 3
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_0521, #[ind_dop_2121]
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p1 p2 p3])

#Gaston.save(term = "cairolatex", output = "del_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex del_resids.tex`)

In [ ]:
tmargin_ths_plt = 0.5 - tmargin_bmargin_gap
bmargin_ths_plt = 0.05

Gaston.set(preamble="unset grid")
p4 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf="set rmargin at screen $rmargin_ths_plt; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = "format '%d/%b/%y' time (\"01/01/05\",\"01/01/06\")",
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false,#2,
        ylabel = "'Doppler shift resid. (O-C) [Hz]'",
        border="1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(b)' at graph 0.2,0.9"
        #title = "'Doppler shift resid. (O-C)'"
    ),
    handle = 4
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p5 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen $rmargin_ths_plt+0.05; set rmargin at screen 0.8; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")",
        xrange="[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ytics=false,
        border="1+4",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 5
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p6 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen 0.85; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"03/03/21\",\"11/03/21\")",
        xrange="[\"02/03/21\":\"12/03/21\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #format="y ''",
        ytics=false,
        border="1+4+8",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 6
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_0521,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p4 p5 p6])

#Gaston.save(term = "cairolatex", output = "dop_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex dop_resids.tex`)

In [ ]:
Gaston.plot([p1 p2 p3; p4 p5 p6])

Gaston.save(term = "cairolatex", output = "deldop_residuals_dates.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
run(`pdflatex deldop_residuals_dates.tex`)

In [ ]:
#radec_19_20 = filter(x->x.yr>=2019, radec_0421)
radec_19_20 = filter(x->x.yr>=2019 && ( DateTime(x.yr, x.month, x.day) != DateTime(2021, 1, 28) ), radec_0421)
t_19_20 = 2000 .+ (select(radec_19_20, :dt_utc_obs=>x->datetime2julian(x)) .- J2000) ./ yr
α_19_20_obs = select(radec_19_20, (:α_obs, :α_corr)=>x->x[1]-x[2])
δ_19_20_obs = select(radec_19_20, (:δ_obs, :δ_corr)=>x->x[1]-x[2])
α_19_20_OR6 = select(radec_19_20, :α_comp)(x_OR6)
δ_19_20_OR6 = select(radec_19_20, :δ_comp)(x_OR6)
α_19_20_OR7 = select(radec_19_20, :α_comp)(x_OR7)
δ_19_20_OR7 = select(radec_19_20, :δ_comp)(x_OR7)
res_α_19_20_obs_OR6 = α_19_20_obs .- α_19_20_OR6
res_δ_19_20_obs_OR6 = δ_19_20_obs .- δ_19_20_OR6
res_α_19_20_obs_OR7 = α_19_20_obs .- α_19_20_OR7
res_δ_19_20_obs_OR7 = δ_19_20_obs .- δ_19_20_OR7
;

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv);

In [ ]:
cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_19_20_obs_OR6, res_δ_19_20_obs_OR6)))

In [ ]:
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_19_20_obs_OR7, res_δ_19_20_obs_OR7)))

In [ ]:
ell_3σ_x_OR6 = mean(res_α_19_20_obs_OR6) .+ cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = mean(res_δ_19_20_obs_OR6) .+ cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = mean(res_α_19_20_obs_OR7) .+ cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = mean(res_δ_19_20_obs_OR7) .+ cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
scatter(res_α_19_20_obs_OR6, res_δ_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!(res_α_19_20_obs_OR7, res_δ_19_20_obs_OR7, msc=:auto, m=:cross, ms=3.0, label="OR7")
plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, color=1, label=false)
plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, color=2, label=false)
scatter!([mean(res_α_19_20_obs_OR6)], [mean(res_δ_19_20_obs_OR6)], m=:circle, color=3, ms=5.0, label=false)
scatter!([mean(res_α_19_20_obs_OR7)], [mean(res_δ_19_20_obs_OR7)], m=:circle, color=4, ms=5.0, label=false)
xlabel!("αcos(δ) [arcsec]")
ylabel!("δ [arcsec]")

In [ ]:
scatter(t_19_20, res_α_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!(t_19_20, res_α_19_20_obs_OR7, msc=:auto, m=:cross, ms=3.0, label="OR7")
xlabel!("Year")
ylabel!("αcos(δ) [arcseconds]")

In [ ]:
scatter(t_19_20, res_δ_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!(t_19_20, res_δ_19_20_obs_OR7, msc=:auto, m=:cross, ms=3.0, label="OR7")
xlabel!("2020 month")
ylabel!("δ [arcseconds]")

In [ ]:
x0_obs = select(del_2121, :τ_obs)[end-1]
y0_obs = select(dop_2121, :ν_obs)[1]

xTN = x0_obs - res_τ[end-1] # O-(O-C) = C
yTN = y0_obs - res_ν[end] # O-(O-C) = C

@show xTN(x_OR6_v15), yTN(x_OR6_v15)
@show xTN(x_OR7_v15), yTN(x_OR7_v15)
@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7_v15 = Hermitian(t_x02xyTN(x_OR7_v15)'*Γ_OR7_v15*(t_x02xyTN(x_OR7_v15)))
Γ_xyTN_OR6_v15 = Hermitian(t_x02xyTN(x_OR6_v15)[1:6,:]'*Γ_OR6_v15*(t_x02xyTN(x_OR6_v15)[1:6,:]))
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7_v15 = cholesky(Γ_xyTN_OR7_v15).L
L_xyTN_OR6_v15 = cholesky(Γ_xyTN_OR6_v15).L
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6_v15))
@show sqrt.(diag(Γ_xyTN_OR7_v15))
@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6_v15 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7_v15 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6_v15[i,:] = L_xyTN_OR6_v15*randn(2)
    xyTN_v_OR7_v15[i,:] = L_xyTN_OR7_v15*randn(2)
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv)

xTN_3σ_ell_OR6_v15 = (L_xyTN_OR6_v15[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6_v15[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6_v15 = (L_xyTN_OR6_v15[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6_v15[2,2]*rndv_2sph_yTN)

xTN_3σ_ell_OR7_v15 = (L_xyTN_OR7_v15[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7_v15[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7_v15 = (L_xyTN_OR7_v15[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7_v15[2,2]*rndv_2sph_yTN)

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)

xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN)

ΔxTN_OR6_v15 = 0.0
ΔyTN_OR6_v15 = 0.0

ΔxTN_OR7_v15 = xTN(x_OR7_v15) - xTN(x_OR6_v15)
ΔyTN_OR7_v15 = yTN(x_OR7_v15) - yTN(x_OR6_v15)

ΔxTN_OR6 = xTN(x_OR6) - xTN(x_OR6_v15)
ΔyTN_OR6 = yTN(x_OR6) - yTN(x_OR6_v15)

ΔxTN_OR7 = xTN(x_OR7) - xTN(x_OR6_v15)
ΔyTN_OR7 = yTN(x_OR7) - yTN(x_OR6_v15)

ΔxTN_obs = x0_obs - xTN(x_OR6_v15)
ΔyTN_obs = y0_obs - yTN(x_OR6_v15)

p1 = Gaston.plot([ΔxTN_OR6_v15], [ΔyTN_OR6_v15], w=:p, lw=5, lc=1, title="'OR6'",
    Axes(
        xlabel="'Delay offset [us]'",
        ylabel="'Doppler offset [Hz]'",
        xrange=(-1300,400),
        xtics=400,
        yrange=(-5,15),
    ),
    handle=1
)
Gaston.plot!(ΔxTN_OR6_v15 .+ xTN_3σ_ell_OR6_v15, ΔyTN_OR6_v15 .+ yTN_3σ_ell_OR6_v15, w=:l, lw=5, lc=1)

Gaston.plot!([ΔxTN_OR7_v15], [ΔyTN_OR7_v15], w=:p, lw=5, lc=2, title="'OR7'")
Gaston.plot!(ΔxTN_OR7_v15 .+ xTN_3σ_ell_OR7_v15, ΔyTN_OR7_v15 .+ yTN_3σ_ell_OR7_v15, w=:l, lw=5, lc=2)

Gaston.plot!([ΔxTN_OR6], [ΔyTN_OR6], w=:p, lc=3, title="'OR6 + 2021 data'")
Gaston.plot!(ΔxTN_OR6 .+ xTN_3σ_ell_OR6, ΔyTN_OR6 .+ yTN_3σ_ell_OR6, w=:l, lw=5, lc=3)

Gaston.plot!([ΔxTN_OR7], [ΔyTN_OR7], w=:p, lc=4, title="'OR7 + 2021 data'")
Gaston.plot!(ΔxTN_OR7 .+ xTN_3σ_ell_OR7, ΔyTN_OR7 .+ yTN_3σ_ell_OR7, w=:l, lw=5, lc=4)
Gaston.plot!([ΔxTN_obs], [ΔyTN_obs], supp=[[1.0] [0.4]], w=:xyerrorbars, pt=9, lw=5, lc=6, title="'observed'")

p2 = Gaston.plot([ΔxTN_OR7_v15], [ΔyTN_OR7_v15], w=:p, pt=3, lw=5, lc=2, title="'OR7'", handle=2,
    Axes(
        xlabel="'Delay offset [us]'", 
        ylabel="'Doppler offset [Hz]'",
        xrange=(-256,-246),
        yrange=(1.0,4.0)
        )
)
Gaston.plot!(ΔxTN_OR7_v15 .+ xTN_3σ_ell_OR7_v15, ΔyTN_OR7_v15 .+ yTN_3σ_ell_OR7_v15, w=:l, lw=5, lc=2)

Gaston.plot!([ΔxTN_OR6], [ΔyTN_OR6], w=:p, pt=5, lc=3, title="'OR6 + 2021 data'")
Gaston.plot!(ΔxTN_OR6 .+ xTN_3σ_ell_OR6, ΔyTN_OR6 .+ yTN_3σ_ell_OR6, w=:l, lw=5, lc=3)
Gaston.plot!([ΔxTN_OR7], [ΔyTN_OR7], w=:p, pt=7, lc=4, title="'OR7 + 2021 data'")
Gaston.plot!(ΔxTN_OR7 .+ xTN_3σ_ell_OR7, ΔyTN_OR7 .+ yTN_3σ_ell_OR7, w=:l, lw=5, lc=4)
Gaston.plot!([ΔxTN_obs], [ΔyTN_obs], supp=[[1.0] [0.4]], w=:xyerrorbars, pt=9, lw=5, lc=6, title="'observed'")

Gaston.plot([p1 p2])

#Gaston.save(term = "cairolatex", output = "deldop_2021_predobs.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex deldop_2021_predobs.tex`)

In [ ]:
σ_dop_2121[1]

In [ ]:
σ_del_2121[1]

In [ ]:
deldop_2121[end].τ_obs

In [ ]:
select(del_2121, :τ_obs)[1]

In [ ]:
select(del_2121, :τ_obs)[end-1]

In [ ]:
select(dop_2121, :ν_obs)[1]

In [ ]:
xTN(x_OR6_v15)

In [ ]:
Gaston.set(preamble="set grid; set yrange [-6:6]")

p1 = Gaston.plot(
    del_dates_plot[ind_del_0506],
    res_τ(x_OR7)[ind_del_0506],
    supp = σ_del_0521[ind_del_0506],
    curveconf = "u 1:3:4 w errorbars pt 4 lw 2",
    Axes(
        linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics="format '%d/%b/%y' time \"28/01/05\", 3*86400, \"31/01/05\"",
        xrange="[\"27/01/05\":\"01/02/05\"]",
        xlabel  = "\"Date\"",
        bars = 2,
        ylabel = "'\$[\$us\$]\$'",
        #title = "'Time delay resid. (O-C)'"
    ),
    handle = 1
)
Gaston.plot!(
    del_dates_plot[ind_del_0506],
    res_τ(x_OR6)[ind_del_0506],
    supp = σ_del_0521[ind_del_0506],
    curveconf = "u 1:3:4 w errorbars pt 6 lw 2",
    lc = 2
)

p2 = Gaston.plot(
    del_dates_plot[ind_del_1213],
    res_τ(x_OR7)[ind_del_1213],
    supp = σ_del_0521[ind_del_1213],
    curveconf = "u 1:3:4 w errorbars pt 6 lw 2",
    Axes(
        linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics="format '%d/%b/%y' time (\"01/01/13\",\"01/03/13\")",
        xrange="[\"15/12/12\":\"20/03/13\"]",
        xlabel  = "\"Date\"",
        bars = 2,
        ylabel = "'\$[\$us\$]\$'",
    ),
    handle = 2
)
Gaston.plot!(
    del_dates_plot[ind_del_1213],
    res_τ(x_OR6)[ind_del_1213],
    supp = σ_del_0521[ind_del_1213],
    curveconf = "u 1:3:4 w errorbars pt 6 lw 2",
    lc = 2
)

p3 = Gaston.plot(
    del_dates_plot[ind_del_2121],
    res_τ(x_OR7)[ind_del_2121],
    supp = σ_del_0521[ind_del_2121],
    curveconf = "u 1:3:4 w errorbars pt 6 lw 2",
    Axes(
        linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics="format '%d/%b/%y' time (\"01/03/21\",\"20/03/21\")",
        xrange="[\"25/02/21\":\"25/03/21\"]",
        xlabel  = "\"Date\"",
        bars = 2,
        ylabel = "'\$[\$us\$]\$'",
    ),
    handle = 3
)
Gaston.plot!(
    del_dates_plot[ind_del_2121],
    res_τ(x_OR6)[ind_del_2121],
    supp = σ_del_0521[ind_del_2121],
    curveconf = "u 1:3:4 w errorbars pt 6 lw 2",
    lc = 2
)

Gaston.plot([p1 p2 p3])

#Gaston.save(term = "cairolatex", output = "deltest.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex deltest.tex`)